# 토픽 모형


## # LDA

# @ 자료 수집

In [1]:
import requests 
import lxml.html

In [ ]:
url = 'http://search.daum.net/search?p={}&w=news&cluster=n&q=인공지능&sort=recency&DA=PGD&sd=20180402000000&ed=20180408235959&period=u'

In [ ]:
urls = []
for page in range(1, 251):
    res = requests.get(url.format(page))
    root = lxml.html.fromstring(res.text)
    for link in root.cssselect('a.f_nb'):
        urls.append(link.attrib['href'])

In [ ]:
len(urls)

In [ ]:
articles = []
for u in tqdm.tqdm_notebook(urls):
    if not u.startswith('http'):
        continue
    res = requests.get(u)
    root = lxml.html.fromstring(res.text)
    body = root.cssselect('.article_view').pop()
    content = body.text_content().strip()  # 본문을 가져와 앞뒤 공백을 제거
    articles.append(content)

In [ ]:
import pandas
df = pandas.DataFrame({'article': articles})
df.to_excel('daum_news_ai.xlsx')

---------

# @ 데이터 불러오기

In [2]:
import pandas

In [5]:
df = pandas.read_excel('data/daum_news_ai.xlsx')

In [6]:
df.head()

,article
0,"Elon Musk, the visionary entrepreneur, fired a..."
1,"배달앱 ‘배달의민족’, 프리미엄 외식 배달 서비스 ‘배민라이더스’ 등을 운영하는 ‘..."
2,[서울경제] 서울경제신문이 4차 산업혁명 시대에 필요한 인재 육성을 위한 교육혁신 ...
3,[경향신문] 인공지능 스피커를 몇 개 구해서 집과 사무실에 연결해 두었다. 친구 삼...
4,의료분야에도 4차 산업혁명 바람이 거세다. 최근 등장한 인공지능(AI)·빅데이터와 ...


# @ 태깅

In [8]:
from konlpy.tag import Twitter

In [9]:
tagger = Twitter()

In [11]:
def kor_noun(text):
    words = []
    for w in tagger.nouns(text):
        if len(w) > 1:
            words.append(w)
    return words

In [12]:
kor_noun('한글은 한국어를 표현하는 문자인 것이다.')

['한글', '한국어', '문자']

### # CountVectorizer

In [13]:
from sklearn.feature_extraction.text import CountVectorizer

In [15]:
cv = CountVectorizer(tokenizer=kor_noun, max_features=1000)

In [16]:
articles = df['article'].astype(str)

In [17]:
tdm = cv.fit_transform(articles)

In [18]:
words = cv.get_feature_names()

In [19]:
sorted(zip(tdm.sum(axis=0).flat, words), reverse=True)[:20]

[(4664, '점수'),
 (3801, '투자'),
 (3748, '기록'),
 (2948, '종목'),
 (2700, '지능'),
 (2545, '인공'),
 (2396, '기자'),
 (2368, '기업'),
 (2083, '시장'),
 (1981, '진단'),
 (1940, '기술'),
 (1890, '업종'),
 (1768, '상위'),
 (1746, '한국'),
 (1513, '최근'),
 (1512, '상장'),
 (1494, '수익률'),
 (1466, '전체'),
 (1452, '라이온'),
 (1444, '그림')]

### # TF-IDF

In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [34]:
cv = TfidfVectorizer(tokenizer=kor_noun, max_features=1000)

In [35]:
articles = df['article'].astype(str)

In [36]:
tdm = cv.fit_transform(articles)

In [37]:
words = cv.get_feature_names()

In [38]:
sorted(zip(tdm.sum(axis=0).flat, words), reverse=True)[:20]

[(198.18993182748153, '점수'),
 (144.75822317611525, '기록'),
 (131.15521788332396, '투자'),
 (124.26033078340178, '종목'),
 (83.054047150632428, '진단'),
 (77.075724989774741, '업종'),
 (75.804661342717964, '상위'),
 (65.936053756202966, '기업'),
 (63.625949007361385, '상장'),
 (63.605531348506148, '수익률'),
 (62.927439223576535, '라이온'),
 (61.043406154138069, '그림'),
 (58.501881747678446, '종합'),
 (58.088455427563296, '전체'),
 (57.4347625143578, '시장'),
 (50.658042502085692, '기자'),
 (48.531675740085312, '최근'),
 (46.214242321339917, '성장'),
 (45.893100789089658, '지능'),
 (45.76222348748869, '한국')]

------------_

In [20]:
from gensim.matutils import Sparse2Corpus

corpus = Sparse2Corpus(tdm.T)

In [21]:
from gensim.models.ldamodel import LdaModel
lda = LdaModel(corpus=corpus, id2word=dict(enumerate(words)),
               num_topics=10, passes=10, iterations=50,
               random_state=1234)

ex) num_topics : 기사 1,000개 => 주제 10개

In [23]:
lda.show_topic(0)

[('점수', 0.060508076110892116),
 ('기록', 0.047183343674121414),
 ('투자', 0.043451852000209269),
 ('종목', 0.037737527315098579),
 ('진단', 0.025403065481961376),
 ('업종', 0.023931418436736032),
 ('상위', 0.022801779585269119),
 ('기업', 0.02087524526041562),
 ('상장', 0.019187434391780188),
 ('기자', 0.018910169415263748)]

In [24]:
lda.show_topic(1)

[('반도체', 0.06718254613484588),
 ('시장', 0.03178420663611032),
 ('투자', 0.026654382193444959),
 ('지난해', 0.024130539634868404),
 ('호황', 0.021008793485102858),
 ('수출', 0.020826342344318207),
 ('수요', 0.018894187630900412),
 ('메모리', 0.01479712581130526),
 ('성장', 0.013627953691853807),
 ('비중', 0.013470502641745687)]

In [25]:
articles[50]

'[서울경제] \n          삼성전자(005930) 갤럭시 스마트폰의 음악서비스인 ‘삼성 뮤직’을 삼성 스마트TV를 통해서도 이용할 수 있게 된다. \n         \n         삼성전자는 2016년 이후 출시된 타이젠 OS(운영체제) 기반 스마트 TV에서 삼성 뮤직을 서비스한다고 8일 밝혔다. 스마트TV용 삼성 뮤직 애플리케이션을 다운받으면 라디오 스테이션 듣기와 스트리밍 음악 듣기 기능이 제공된다. MP3 다운로드, 유료상품 구매는 불가능하다. 인공지능AI) 플랫폼 ‘빅스비’가 탑재된 2018년형 스마트TV를 이용할 경우 “삼성 뮤직에서 ○○○ 음악 틀어줘”와 같은 음성 명령도 가능하다. 삼성전자 측은 “모바일과 TV에서 동일한 계정으로 삼성 뮤직을 쓰고 즐겨찾기 등도 연동해 이용할 수 있다”고 밝혔다. /권경원기자 nahere@sedaily.com'

In [26]:
def tdm2doc(tdm, i):
    doc = []
    for i, n in enumerate(tdm[i].toarray().flat):
        if n > 0:
            doc.append((i, n))
    return doc

In [27]:
doc = tdm2doc(tdm, 50)

In [28]:
doc

[(29, 1),
 (48, 1),
 (51, 1),
 (97, 1),
 (130, 1),
 (133, 1),
 (139, 1),
 (270, 1),
 (389, 9),
 (397, 1),
 (408, 2),
 (409, 1),
 (463, 4),
 (464, 1),
 (523, 1),
 (544, 2),
 (618, 1),
 (628, 1),
 (631, 3),
 (659, 1),
 (661, 1),
 (727, 3),
 (746, 1),
 (794, 1),
 (852, 1),
 (857, 1),
 (878, 1),
 (925, 1)]

In [29]:
lda.get_document_topics(doc)

[(2, 0.2284575106133345),
 (4, 0.034491793348329546),
 (5, 0.32588335753237968),
 (6, 0.39812181552500392)]

In [30]:
for t in range(10):
    topic = [word for word, p in lda.show_topic(t)]
    print(t, ' '.join(topic))

0 점수 기록 투자 종목 진단 업종 상위 기업 상장 기자
1 반도체 시장 투자 지난해 호황 수출 수요 메모리 성장 비중
2 지능 인공 스마트 기술 시티 중국 통해 기자 시장 사업
3 서울 시민 서울시 창업 혁신 도시 교육 안철수 미래 기술
4 택시 서비스 호출 카카오 고객 국토부 배차 모빌리티 암웨이 유료
5 서비스 기술 체인 정보 블록 냉장고 기반 시스템 기능 추천
6 전자 삼성 분기 사업 시장 실적 투자 억원 영업 스마트폰
7 연구 무기 지능 로봇 카이스트 인공 시스템 개발 인간 기술
8 기업 부회장 삼성 정부 교육 지원 지역 사업 억원 창업
9 기술 센터 지능 인공 텔레콤 게임 데이터 개발 구글 미국


*** 어디다 쓰느냐?

기사별로 주제의 비율의 평균을 확인 
퍼센트를 알 수 있다
주별로 통계를 내면 어떤 시기에 어떤 주제를 변화하는지 
언론사 별로 어떤 주제에 관심이 있는지